In [1]:
import sys
import os
import numpy as np
from itertools import product
import matplotlib.pyplot as plt

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
module_path = os.path.abspath(
    os.path.join(
        "/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control"
    )
)
if module_path not in sys.path:
    sys.path.append(module_path)

from correlated_noise_q_env_config import (
    q_env_config as gate_q_env_config,
    circuit_context,
    circuit_gate_times,
)
    
from correlated_noise_q_env_config_function import setup_quantum_environment

from quantumenvironment import QuantumEnvironment
from context_aware_quantum_environment import ContextAwareQuantumEnvironment
from hyperparameter_optimization_ressource_constraint import HyperparameterOptimizer
from gymnasium.spaces import Box
from gymnasium.wrappers import RescaleAction, ClipAction
from helper_functions import get_baseline_fid_from_phi_gamma

import logging

logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s INFO %(message)s",  # hardcoded INFO level
    datefmt="%Y-%m-%d %H:%M:%S",
    stream=sys.stdout,
)

/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/venv_qiskit10/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



 NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx', 'unitary']
  Instructions with noise: ['custom_kron(rx,ident)_gate']
  Qubits with noise: [0, 1]
  Specific qubit errors: [('custom_kron(rx,ident)_gate', (0, 1))] 

Circuit context
     ┌─────────────────────────────┐     
q_0: ┤0                            ├──■──
     │  custom_kron(rx,ident)_gate │┌─┴─┐
q_1: ┤1                            ├┤ X ├
     └─────────────────────────────┘└───┘


/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/venv_qiskit10/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/venv_qiskit10/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [2]:
def get_total_runtime_hpo(phis, gammas, max_runtime_per_trial, num_hpo_trials):
    return len(phis) * len(gammas) * max_runtime_per_trial * num_hpo_trials

### Adjust action space for larger noise profiles

In [3]:
gate_q_env_config.action_space

Box(-0.1, 0.1, (7,), float32)

In [4]:
def scale_action_space(phi_gamma_tuple, initial_phi=np.pi, initial_gamma=0.05, initial_space=Box(-0.1, 0.1, (7,), np.float32)):
    
    phi, gamma = phi_gamma_tuple
    
    # Calculate the initial and new ratios
    initial_ratio = initial_phi * initial_gamma
    new_ratio = phi * gamma

    # Scale the action space based on the ratio
    scale_factor = new_ratio / initial_ratio
    new_low = initial_space.low * scale_factor
    new_high = initial_space.high * scale_factor

    # Create a new action space with the scaled values
    new_space = Box(new_low, new_high, initial_space.shape, initial_space.dtype)

    return new_space

In [9]:
phi_gamma_tuple = (np.pi/4, 0.15)
scale_action_space(phi_gamma_tuple, initial_phi=np.pi, initial_gamma=0.05, initial_space=Box(-0.1, 0.1, (7,), np.float32))

Box(-0.075, 0.075, (7,), float32)

In [6]:
def perform_hpo_noisy_study(phis, gammas, max_runtime_per_trial, num_hpo_trials, target_fidelities, lookback_window, experimental_penalty_weights):

    for phi_gamma_tuple in product(phis, gammas):

        gate_q_env_config, circuit_context, _ = setup_quantum_environment(phi_gamma_tuple=phi_gamma_tuple)
        gate_q_env_config.action_space = scale_action_space(phi_gamma_tuple=phi_gamma_tuple)

        print(gate_q_env_config.action_space)

        q_env = ContextAwareQuantumEnvironment(gate_q_env_config, circuit_context)
        q_env = ClipAction(q_env)
        q_env = RescaleAction(q_env, -1.0, 1.0)


        current_dir = os.getcwd()
        grand_parent_dir = os.path.dirname(os.path.dirname(current_dir))

        path_agent_config = os.path.join(os.path.dirname(grand_parent_dir), "agent_config.yaml")
        path_hpo_config = os.path.join(current_dir, "noise_hpo_config.yaml")
        save_results_path = os.path.join("hpo_results", "resource_constraint")


        optimizer = HyperparameterOptimizer(
            q_env=q_env,
            path_agent_config=path_agent_config,
            path_hpo_config=path_hpo_config,
            save_results_path=save_results_path,
            experimental_penalty_weights=experimental_penalty_weights,
            log_progress=False,
        )

        best_trial = optimizer.optimize_hyperparameters(
            num_hpo_trials = num_hpo_trials,
            phi_gamma_tuple = phi_gamma_tuple,
            target_fidelities = target_fidelities,
            lookback_window = lookback_window,
            max_runtime = max_runtime_per_trial
        )

In [7]:
phis = np.pi * np.array([0.25]) # , 0.5, 0.75, 1.0])
gammas = np.linspace(0.15, 0.151, 1)

target_fidelities = [1.0-infid for infid in [1e-3, 1e-4, 1e-5]]

lookback_window = 20

max_runtime_per_trial = 600
num_hpo_trials = 5

experimental_penalty_weights = {
    'penalty_n_shots': 0.01,
    'penalty_per_missed_fidelity': 5e3,
    'fidelity_reward': 1e4,
}

print(f'(Worst Case): Total runtime for HPO: {round(get_total_runtime_hpo(phis, gammas, max_runtime_per_trial, num_hpo_trials)/3600, 4)} hours')

(Worst Case): Total runtime for HPO: 0.8333 hours


TODO: Parallelize workflow across multiple parameter

In [8]:
perform_hpo_noisy_study(phis, gammas, max_runtime_per_trial, num_hpo_trials, target_fidelities, lookback_window, experimental_penalty_weights)

Total Updates applied: 1300
Total Shots applied: 295,685,148
probabilities renormalized
Fidelity stored 0.9994318614482254
Sending Estimator job...
SparsePauliOp(['II', 'IY', 'IZ', 'XI', 'XY', 'XZ'],
              coeffs=[ 0.5       +0.j,  0.70710678+0.j, -0.70710678+0.j,  0.5       +0.j,
  0.70710678+0.j, -0.70710678+0.j])
Finished Estimator job
Total Updates applied: 1301
Total Shots applied: 295,703,298
Fidelity stored 0.9994787633729115
Sending Estimator job...
SparsePauliOp(['II', 'IZ', 'XY', 'YX', 'ZI', 'ZZ'],
              coeffs=[ 0.5       +0.j, -0.70710678+0.j,  0.70710678+0.j, -0.70710678+0.j,
  0.70710678+0.j, -0.5       +0.j])
Finished Estimator job
Total Updates applied: 1302
Total Shots applied: 295,725,078
probabilities renormalized
Fidelity stored 0.9994499919744925
Sending Estimator job...
SparsePauliOp(['II', 'IZ', 'XY', 'YI', 'YZ', 'ZX'],
              coeffs=[ 0.5       +0.j, -0.70710678+0.j,  0.70710678+0.j, -0.70710678+0.j,
  0.5       +0.j, -0.70710678+0.j])
Fin

[I 2024-04-09 18:49:57,865] Trial 4 finished with value: -4390.504034605629 and parameters: {'MINIBATCH_SIZE': 10, 'BATCHSIZE_MULTIPLIER': 11, 'N_SHOTS': 11, 'SAMPLE_PAULIS': 34}. Best is trial 2 with value: -5774.904305861553.


2024-04-09 18:49:57 INFO Best configuration saved to hpo_results/resource_constraint/phi-0.25pi_gamma-0.15_maxruntime-600_custom-cost-value--5774.904306_timestamp_09-04-2024_18-49-57.pickle


In [ ]:
import pickle
with open('/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/template_configurations/qiskit/gate_level/noisy_circuit/hpo_results/resource_constraint/phi-0.5pi_gamma-0.02_maxruntime-600_custom-cost-value-22901.12_timestamp_04-04-2024_16-16-00.pickle', 'rb') as f:
    results = pickle.load(f)

In [ ]:
results['training_results']['fidelity_info']

In [ ]:
results['hyper_params']

In [ ]:
phi = np.pi / 2
gamma = 0.025

gate_q_env_config, circuit_context, circuit_gate_times = setup_quantum_environment(phi_gamma_tuple=(phi, gamma))

In [ ]:
q_env = ContextAwareQuantumEnvironment(gate_q_env_config, circuit_context)
q_env = ClipAction(q_env)
q_env = RescaleAction(q_env, -1.0, 1.0)

q_env.unwrapped.backend

In [ ]:
q_env.unwrapped.action_space

In [ ]:
q_env.unwrapped.circuit_truncations[0].draw("mpl")

In [ ]:
current_dir = os.getcwd()
grand_parent_dir = os.path.dirname(os.path.dirname(current_dir))

path_agent_config = os.path.join(os.path.dirname(grand_parent_dir), "agent_config.yaml")
path_hpo_config = os.path.join(current_dir, "noise_hpo_config.yaml")
save_results_path = os.path.join("hpo_results", "resource_constraint")

In [ ]:
# experimental_penalty_weights = {
#     "runtime": 0.001, # runtime is between 300 and 600 seconds
#     "n_shots": 0.005, # shots are between 1 and 20
#     "batchsize": 0.003, # batchsize is between 1 and 10
#     "sample_paulis": 0.005, # sample_paulis is between 20 and 120
# }

experimental_penalty_weights = {
    'n_shots': 0.01,
    'fidelity_not_achieved': 10,
}

In [ ]:
optimizer = HyperparameterOptimizer(
    q_env=q_env,
    path_agent_config=path_agent_config,
    path_hpo_config=path_hpo_config,
    save_results_path=save_results_path,
    experimental_penalty_weights=experimental_penalty_weights,
    log_progress=False,
)

In [ ]:
max_runtime = 100

In [ ]:
target_fidelities = [1.0-infid for infid in [1e-3, 1e-4, 1e-5]]
target_fidelities

In [ ]:
baseline_fid = get_baseline_fid_from_phi_gamma((phi, gamma))
baseline_fid

In [ ]:
best_trial = optimizer.optimize_hyperparameters(
    num_hpo_trials = 2,
    phi_gamma_tuple = (phi, gamma),
    target_fidelities = target_fidelities,
    max_runtime = max_runtime
)

In [ ]:
best_trial['training_results']['fidelity_history']

In [ ]:
best_trial['training_results']['fidelity_info']

In [ ]:
np.cumsum(q_env.unwrapped.total_shots)[best_trial['training_results']['fidelity_info'][0.999]['update_at']-1]

In [ ]:
np.cumsum(q_env.unwrapped.total_shots)[0]